# 2. Etiquetado de Secuencias: Parte 2

Usaremos los scripts train.py y eval.py del práctico 1:

In [1]:
%run ../../tagging/scripts/train.py --help

Train a sequence tagger.

Usage:
  train.py [options] -c <path> -o <file>
  train.py -h | --help

Options:
  -m <model>    Model to use [default: badbase]:
                  badbase: Bad baseline
                  base: Baseline
  -c <path>     Ancora corpus path.
  -o <file>     Output model file.
  -h --help     Show this screen.


In [2]:
%run ../../tagging/scripts/eval.py --help

Evaulate a tagger.

Usage:
  eval.py [options] -c <path> -i <file>
  eval.py -h | --help

Options:
  -c <path>     Ancora corpus path.
  -i <file>     Tagging model file.
  -p            Show progress bar.
  -m            Show confusion matrix.
  -h --help     Show this screen.


## 2.1. Baseline Malo: Todo es Sustantivo Singular

Primero "entrenamos" este baseline (no hace nada en realidad):

In [3]:
%run ../../tagging/scripts/train.py -c ../../ancora-3.0.1es -m badbase -o badbase

Luego evaluamos. Usamos la opción '-m' para imprimir la matrz de confusión:

In [4]:
%run ../../tagging/scripts/eval.py -c ../../ancora-3.0.1es -i badbase -m


Accuracy: 12.65% / 0.00% / 12.65%

g \ m	sp000	nc0s000	da0000	aq0000	fc	nc0p000	rg	np00000	fp	cc
sp000	-	14.39	-	-	-	-	-	-	-	-	
nc0s000	-	12.65	-	-	-	-	-	-	-	-	
da0000	-	9.70	-	-	-	-	-	-	-	-	
aq0000	-	7.28	-	-	-	-	-	-	-	-	
fc	-	5.85	-	-	-	-	-	-	-	-	
nc0p000	-	5.53	-	-	-	-	-	-	-	-	
rg	-	3.73	-	-	-	-	-	-	-	-	
np00000	-	3.58	-	-	-	-	-	-	-	-	
fp	-	3.55	-	-	-	-	-	-	-	-	
cc	-	3.41	-	-	-	-	-	-	-	-	


La matriz de confusión muestra en la fila X columna Y la proporción de veces que el tagger eligió el tag Y pero el tag verdadero es X. Sólo se imprime la matriz para los 10 tags más frecuentes.

## 2.2. Baseline (Menos Malo): Etiqueta Más Frecuente

**Primero resolver ejercicio 2 del práctico!**

In [1]:
%run ../../tagging/scripts/train.py -c ../../ancora-3.0.1es -m base -o base

In [2]:
%run ../../tagging/scripts/eval.py -c ../../ancora-3.0.1es -i base -m


Accuracy: 87.58% / 95.27% / 18.01% (total / known / unk)

g \ m	sp000	nc0s000	da0000	aq0000	fc	nc0p000	rg	np00000	fp	cc
sp000	14.28	0.05	-	-	-	-	0.01	-	-	-	
nc0s000	0.00	12.22	-	0.25	-	0.00	0.03	0.00	-	0.00	
da0000	-	0.15	9.54	-	-	-	-	-	-	-	
aq0000	0.01	2.05	-	4.84	-	0.13	0.00	-	-	-	
fc	-	-	-	-	5.85	-	-	-	-	-	
nc0p000	-	1.24	-	0.20	-	4.09	-	-	-	-	
rg	0.02	0.31	-	0.04	-	-	3.27	-	-	0.02	
np00000	0.00	2.05	-	0.00	-	0.00	-	1.52	-	0.00	
fp	-	-	-	-	-	-	-	-	3.55	-	
cc	0.00	0.01	-	-	-	-	0.05	0.00	-	3.34	


**Se puede ver que uno de los principales errores del tagger es confundir nombre propio (np00000) con nombre común (nc0s000), con un 2.05% del error total.**

## 2.3. Análisis de Errores

El script de evaluación eval.py calcula variables útiles para hacer análisis de errores:

- **error_count:** matriz de confusión completa.
- **error_sents:** matriz de índices de las oraciones en las que ocurre cada error.

Por ejemplo, podemos ver algunas oraciones en las que el tagger confunde nombre propio con nombre común:

In [7]:
error_count['np00000']['nc0s000']  # / total da 2.05%

1939

In [44]:
sorted(error_sents['np00000']['nc0s000'])[:10]

[0, 1, 4, 5, 6, 7, 8, 9, 12, 13]

Veamos la primer oración:

In [48]:
sent = sents[0]
words, tags = zip(*sent)  # separar las palabras de los tags
sent[:10]

[('El', 'da0000'),
 ('presidente', 'nc0s000'),
 ('del', 'sp000'),
 ('órgano', 'nc0s000'),
 ('regulador', 'aq0000'),
 ('de', 'sp000'),
 ('las', 'da0000'),
 ('Telecomunicaciones', 'np00000'),
 ('se', 'p0000000'),
 ('mostró', 'vmis000')]

In [49]:
model.tag(words[:10])

['da0000',
 'nc0s000',
 'sp000',
 'nc0s000',
 'aq0000',
 'sp000',
 'da0000',
 'nc0s000',
 'p0000000',
 'vmis000']

In [50]:
model.unknown('Telecomunicaciones')

True

Es claro que el tagger se puede mejorar para palabras desconocidas mirando características como el uso de mayúsculas y también de caracteres no estándar del inglés (e.g. letras acentuadas).